<a href="https://colab.research.google.com/github/StefaniaGutu/OffensiveLanguageClassification/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
import string
import spacy
import pickle
from os.path import exists
from gensim.models import Word2Vec as w2v
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_csv('sample_data/labeled_data.csv')
print(df)

FileNotFoundError: ignored

In [ ]:
dataset_labels = df["class"]
dataset_tweets = df["tweet"]

Preprocesare

In [ ]:
def preprocess(dataset):
    # remove @user
    tweets_no_user = [re.sub(r'@[\w]*', '', tweet) for tweet in dataset_tweets]

    # remove URL
    tweets_no_links = [re.sub(r'http\S+', '', tweet) for tweet in tweets_no_user]

    # remove hashtags
    tweets_no_hashtags = [re.sub(r'#[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_links]

    # remove &_;
    tweets_no_apersand = [re.sub(r'&[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_hashtags]

    # remove Punctuations and Special Characters
    tweets_no_punct = [tweet.translate(str.maketrans('', '', string.punctuation)) for tweet in tweets_no_apersand]

    # remove Numbers
    tweets_no_digits = [re.sub(r'\d+', '', tweet) for tweet in tweets_no_punct]

    # Convert to lower case
    tweets_lower = [tweet.lower() for tweet in tweets_no_digits]

    # Convert more than 2 letter repetitions to 2 letter
    tweets_no_repetitions = [re.sub(r'(.)\1+', r'\1\1', tweet) for tweet in tweets_lower]

    # Replace multiple spaces with a single space
    tweets_no_spaces = [re.sub(r'\s+', ' ', tweet) for tweet in tweets_no_repetitions]

    # Lemma
    nlp = spacy.load('en_core_web_sm')
    all_words_lemma = [[word.lemma_ for word in nlp(sent)] for sent in tweets_no_spaces]

    # remove stopwords
    stop_words_spacy = nlp.Defaults.stop_words
    all_words_without_stops = [[word for word in sent if word not in stop_words_spacy] for sent in all_words_lemma]

    # remove Short Words
    words_no_shortWords = [[word for word in tweet if len(word)>2] for tweet in all_words_without_stops]
    tweets_no_shortWords = [word for word in words_no_shortWords]

    return tweets_no_shortWords

In [ ]:
preprocessed_dataset = preprocess(dataset_tweets)

print(preprocessed_dataset[:5])

[['woman', 'complain', 'clean', 'house', 'man', 'trash'], ['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place'], ['dawg', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit'], ['look', 'like', 'tranny'], ['shit', 'hear', 'true', 'faker', 'bitch', 'tell']]


In [ ]:
preprocessed_dataset[:15]

[['woman', 'complain', 'clean', 'house', 'man', 'trash'],
 ['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place'],
 ['dawg', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit'],
 ['look', 'like', 'tranny'],
 ['shit', 'hear', 'true', 'faker', 'bitch', 'tell'],
 ['shit', 'blow', 'meclaim', 'faithful', 'somebody', 'fuck', 'hoe'],
 ['sit', 'hate', 'bitch', 'shit'],
 ['cause', 'tired', 'big', 'bitch', 'come', 'skinny', 'girl'],
 ['bitch'],
 ['hobby', 'include', 'fight', 'mariam', 'bitch'],
 ['keek', 'bitch', 'curve', 'lol', 'walk', 'conversation', 'like', 'smh'],
 ['murda', 'gang', 'bitch', 'gang', 'land'],
 ['hoe', 'smoke', 'loser', 'yea'],
 ['bad', 'bitch', 'thing', 'like'],
 ['bitch']]

In [ ]:
# Save preprocessed dataset
pck = open("./pickles/preprocessed_dataset.pck", "wb")
pickle.dump({
    'preprocessed_dataset': preprocessed_dataset,
    'dataset_labels': dataset_labels
}, pck)

In [7]:
preprocessed_dataset = []
dataset_labels = []

pck = open("./pickles/preprocessed_dataset.pck", "rb")
result = pickle.load(pck)
preprocessed_dataset = result['preprocessed_dataset']
dataset_labels = result['dataset_labels']

In [8]:
new_dataset_labels = []
for i in dataset_labels:
  if i == 1:
    new_dataset_labels.append(0)
  elif i == 2:
    new_dataset_labels.append(1)
  else:
    new_dataset_labels.append(i)

print(new_dataset_labels)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 

Vectorizare

In [9]:
w = w2v(
    preprocessed_dataset,
    min_count=3,
    sg=1,  # sg=1 inseamna ca se foloseste skip-gram, sg=0 inseamna ca se foloseste CBOW
    window=7
)

print(w.wv.key_to_index)

emb_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index=w.wv.key_to_index
    )
)

print(emb_df.shape)
emb_df.head()

{'bitch': 0, 'hoe': 1, 'like': 2, 'pussy': 3, 'fuck': 4, 'ass': 5, 'shit': 6, 'nigga': 7, 'trash': 8, 'lol': 9, 'know': 10, 'niggas': 11, 'look': 12, 'love': 13, 'want': 14, 'good': 15, 'girl': 16, 'think': 17, 'man': 18, 'bad': 19, 'talk': 20, 'bird': 21, 'fucking': 22, 'need': 23, 'tell': 24, 'hate': 25, 'faggot': 26, 'come': 27, 'time': 28, 'let': 29, 'day': 30, 'eat': 31, 'real': 32, 'people': 33, 'right': 34, 'white': 35, 'lmao': 36, 'try': 37, 'dick': 38, 'wit': 39, 'stop': 40, 'wanna': 41, 'bout': 42, 'play': 43, 'damn': 44, 'little': 45, 'life': 46, 'niggah': 47, 'dat': 48, 'nigger': 49, 'big': 50, 'cause': 51, 'ill': 52, 'new': 53, 'cunt': 54, 'charlie': 55, 'money': 56, 'fag': 57, 'lil': 58, 'hit': 59, 'feel': 60, 'game': 61, 'ugly': 62, 'boy': 63, 'ghetto': 64, 'use': 65, 'tweet': 66, 'dumb': 67, 'woman': 68, 'twitter': 69, 'fuckin': 70, 'yellow': 71, 'thing': 72, 'guy': 73, 'tho': 74, 'mean': 75, 'stupid': 76, 'work': 77, 'way': 78, 'watch': 79, 'start': 80, 'black': 81, 'a

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
bitch,-0.168428,0.254123,0.035317,0.101636,0.169947,-0.268004,0.129109,0.476947,-0.466710,-0.160013,...,0.079504,0.052101,0.243820,-0.052712,0.782214,-0.066776,-0.108570,-0.123751,-0.105677,0.199360
hoe,-0.082139,0.189031,-0.012057,0.062425,0.224028,-0.454158,0.147647,0.309290,-0.399306,-0.287638,...,-0.013086,0.228416,0.152385,-0.022585,0.605049,-0.026519,0.146081,-0.158240,-0.171339,0.088801
like,-0.070313,0.019275,-0.121841,-0.113614,0.214970,-0.218016,0.068430,0.474805,-0.255205,-0.000595,...,-0.010008,0.165130,0.317129,-0.123038,0.342976,0.266176,0.327149,0.026283,0.180547,-0.014064
pussy,-0.215300,0.108474,-0.040753,0.027369,0.214600,-0.312398,0.059587,0.422925,-0.074956,-0.191224,...,-0.008494,0.110299,-0.048905,-0.049460,0.782182,-0.048143,0.184524,-0.110237,-0.228159,0.045824
fuck,-0.093743,0.063026,-0.230708,0.017680,0.107677,-0.286833,0.149237,0.425998,-0.211717,-0.125925,...,0.252870,0.220548,0.065813,0.132006,0.511306,0.036765,0.091001,-0.057923,0.020312,0.040740


In [10]:
new_preprocessed_dataset = [w.wv.get_vector(str(n)) for n in w.wv.key_to_index]
print(len(new_preprocessed_dataset))
print(len(new_dataset_labels))

4893
24783


In [11]:
vectorized_sentences = [] 
vectorized_labels = []

for i, sentence in enumerate(preprocessed_dataset):
    sentence_vectors = []
    # print(sentence)
    for word in sentence:
        try:
            word_vector = w.wv.get_vector(word)
            sentence_vectors.append(word_vector)
            # print(sentence_vectors)
        except KeyError:
            # If a word is not in the vocabulary, ignore it
            pass
    try:
      vectorized_sentence = np.mean(sentence_vectors)
      vectorized_sentences.append(vectorized_sentence.tolist())
      vectorized_labels.append(new_dataset_labels[i])
    except ZeroDivisionError:
      pass

print(vectorized_sentences[:5])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[0.00668934965506196, 0.0027436986565589905, 0.004378610756248236, 0.007089416030794382, 0.003680651541799307]


Split data in train, test, validation

In [12]:
X_train, X_val_test, y_train, y_val_test = train_test_split(vectorized_sentences, vectorized_labels, test_size=0.30, random_state=42)

X_test, X_validation, y_test, y_validation = train_test_split(X_val_test, y_val_test, test_size=0.50, random_state=42)

In [13]:
print(len(X_train))
print(len(X_test))
print(len(X_validation))

17348
3717
3718


Model

In [82]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
X_val_tensor = torch.tensor(X_validation)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(y_train)
y_val_tensor = torch.tensor(y_validation)
y_test_tensor = torch.tensor(y_test)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

In [83]:
# dataloaders
batch_size = 50  # you can change batch size
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [84]:
# define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)  # reshape to (batch_size, sequence_length, input_size)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # get the last output of the sequence
        out = self.fc(out)
        return out

In [85]:
model = RNN(1, 32, 2)  # assuming binary classification (hate speech or not)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [87]:
for epoch in range(100):  # number of epochs
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    # validation
    model.eval()
    val_losses = []
    for inputs, labels in val_loader:
        with torch.no_grad():
            val_outputs = model(inputs.float())
            val_loss = criterion(val_outputs, labels.long())
        val_losses.append(val_loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {sum(val_losses)/len(val_losses)}')

Epoch 1, Loss: nan, Val Loss: nan
Epoch 2, Loss: nan, Val Loss: nan
Epoch 3, Loss: nan, Val Loss: nan
Epoch 4, Loss: nan, Val Loss: nan
Epoch 5, Loss: nan, Val Loss: nan
Epoch 6, Loss: nan, Val Loss: nan
Epoch 7, Loss: nan, Val Loss: nan
Epoch 8, Loss: nan, Val Loss: nan
Epoch 9, Loss: nan, Val Loss: nan
Epoch 10, Loss: nan, Val Loss: nan
Epoch 11, Loss: nan, Val Loss: nan
Epoch 12, Loss: nan, Val Loss: nan
Epoch 13, Loss: nan, Val Loss: nan
Epoch 14, Loss: nan, Val Loss: nan
Epoch 15, Loss: nan, Val Loss: nan
Epoch 16, Loss: nan, Val Loss: nan
Epoch 17, Loss: nan, Val Loss: nan
Epoch 18, Loss: nan, Val Loss: nan
Epoch 19, Loss: nan, Val Loss: nan
Epoch 20, Loss: nan, Val Loss: nan
Epoch 21, Loss: nan, Val Loss: nan
Epoch 22, Loss: nan, Val Loss: nan
Epoch 23, Loss: nan, Val Loss: nan
Epoch 24, Loss: nan, Val Loss: nan
Epoch 25, Loss: nan, Val Loss: nan
Epoch 26, Loss: nan, Val Loss: nan
Epoch 27, Loss: nan, Val Loss: nan
Epoch 28, Loss: nan, Val Loss: nan
Epoch 29, Loss: nan, Val Loss

In [88]:
# testing
model.eval()
correct = 0
total = 0
for inputs, labels in test_loader:
    with torch.no_grad():
        test_outputs = model(inputs.float())
        _, predicted = torch.max(test_outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()
accuracy = correct / total
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.8305084745762712
